In [ ]:
# Converts PIPS data from the raw TOB32 to TOA5 format
# uses external system call to camp2ascii

In [ ]:
import os
import sys
import subprocess
import glob
import re

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [atoi(c) for c in re.split(r'(\d+)', text)]

In [ ]:
basedir = '/Users/dawson29/Dropbox/Projects/TriPIPS'
inputsubdir = 'incoming_092819/'
outputsubdir = 'converted_092819/'
inputdir = os.path.join(basedir, inputsubdir)
outputdir = os.path.join(basedir, outputsubdir)
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
filelist = glob.glob(inputdir+'/*.dat')
filelist.sort(key=natural_keys)
print(filelist)

In [ ]:
# Convert each file one-by-one
for rawfile in filelist:
    outfile = 'TOA5_' + os.path.basename(rawfile)
    outpath = os.path.join(outputdir, outfile)
    if not os.path.exists(outpath):
        print("Processing {}".format(outfile))
        try:
            output = subprocess.run(['camp2ascii', rawfile], stdout=subprocess.PIPE, universal_newlines=True, errors='ignore')
            with open(outpath, 'w') as f:
                f.write(output.stdout)
        except:
            print("Cannot read file! Skipping!")
    else:
        print("Skipping {}".format(outfile))

In [ ]:
# Fix converted one-sec files that have bad data at the end

filelist = glob.glob(outputdir+'/*Met_Location*.dat')

for converted_file in filelist:
    badfile = False
    newlines = []
    with open(converted_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.strip().split(',')
            if "1944" in tokens[0]:
                if not badfile:
                    print("Found bad file {}! Fixing!".format(converted_file))
                badfile = True
                break
            else:
                newlines.append(line)
    if badfile:
        new_file = converted_file + '_fixed'
        with open(new_file, 'w') as f:
            f.writelines(newlines)
    
                